# Gait Video Study 
### OpenPose on our collected video data to extract 12 2D coordinates (hip, knee, ankle, big/small toe and heel) from lower body frames and 8 2D coordinates (ankle, big/small toe and heel) from feet focussed camera frames 

In [2]:
import os
import numpy as np
import sys

In [3]:
dir_path='C:\\Users\\purpl\\Desktop\\openpose-master\\build'
sys.path.append(dir_path + '/python/openpose/Release')
os.environ['PATH']  = os.environ['PATH'] + ';' + dir_path + '/x64/Release;' +  dir_path + '/bin;'

In [4]:
os.environ['PATH']

'C:\\Users\\purpl\\anaconda3;C:\\Users\\purpl\\anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\purpl\\anaconda3\\Library\\usr\\bin;C:\\Users\\purpl\\anaconda3\\Library\\bin;C:\\Users\\purpl\\anaconda3\\Scripts;C:\\Users\\purpl\\anaconda3\\bin;C:\\Users\\purpl\\anaconda3\\condabin;C:\\Users\\purpl\\anaconda3;C:\\Users\\purpl\\anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\purpl\\anaconda3\\Library\\usr\\bin;C:\\Users\\purpl\\anaconda3\\Library\\bin;C:\\Users\\purpl\\anaconda3\\Scripts;C:\\Program Files\\Rockwell Software\\RSCommon;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0;C:\\Windows\\System32\\OpenSSH;C:\\Program Files\\Microsoft SQL Server\\130\\Tools\\Binn;C:\\Program Files\\PuTTY;C:\\Users\\purpl\\AppData\\Local\\Microsoft\\WindowsApps;.;C:\\Users\\purpl\\Desktop\\openpose-master\\build/x64/Release;C:\\Users\\purpl\\Desktop\\openpose-master\\build/bin;'

In [5]:
from openpose import pyopenpose as op
import pandas as pd
import cv2
import time 
import matplotlib.pyplot as plt

#### Function to show original image

In [6]:
def show_original_image(image_path):
    im = cv2.imread(image_path)
    RGB_im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.imshow(RGB_im)
    plt.show()

#### Function to setup OpenPose parameters

In [7]:
#To create all directories for saving the openpose_data 
# for cohort in cohorts:
#     for trial in trials:
#         for camera in cameras:    
#             frame_path = raw_path+cohort+trial+camera #Path to save the frames to 
#             videos = os.listdir(frame_path)
#             for video in videos:
#                 if not os.path.exists(openpose_data+cohort+trial+camera+video):
#                     os.makedirs(openpose_data+cohort+trial+camera+video)

In [8]:
def process_openpose(image_path, output_path, feet):
    # Custom Params (refer to include/openpose/flags.hpp for more parameters)
    params = dict()
    params["model_folder"] = dir_path+"\\..\\models\\"
    params["number_people_max"] = 1
    if feet:
        params["maximize_positives"] = True #For feet images 
        params["number_people_max"] = 1

    # Starting OpenPose
#     print (params["number_people_max"])
    opWrapper = op.WrapperPython()
    opWrapper.configure(params)
    opWrapper.start()
    
    im = cv2.imread(image_path)
    
    # Process Image
    datum = op.Datum()
    datum.cvInputData = im
    opWrapper.emplaceAndPop([datum])

    # Display Image
    RGBOutputData = cv2.cvtColor(datum.cvOutputData, cv2.COLOR_BGR2RGB)
    plt.imshow(RGBOutputData)
    plt.savefig(output_path)
    
    body_keypoints = datum.poseKeypoints
#     print("Body keypoints: \n", body_keypoints)
#     Saving the body keypoints to a .csv file 
    try:
        pd.DataFrame(body_keypoints[0], columns = ['x', 'y', 'confidence']).to_csv(output_path[:-3]+'csv')
    except:
        pass

In [9]:
#Configuration for which to run the code for 
cohorts = ['\\HOA', '\\MS', '\\PD']
trials = ['\\beam_walking', '\\walking']
cameras = ['\\feet\\', '\\lower_body\\']

In [13]:
#To read original images from 
raw_path = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data' 
#To save the openpose processed images to openpose_data folder 
openpose_data = raw_path+'\\..\\openpose_data'
feet_bool = False
for cohort in cohorts:
    for trial in trials:
        for camera in cameras:   
            if camera == '\\feet\\':
                feet_bool = True
            frame_path = raw_path+cohort+trial+camera #Path to save the frames to 
#             print (frame_path)
            videos = os.listdir(frame_path)
#             print (videos)
            for video in videos:
                frames = os.listdir(frame_path+video)
                start_time = time.time()
                for frame in frames:
                    if (frame[-3:]=='jpg'):
                        im_path = frame_path+video+'\\'+frame
    #                     show_original_image(im_path)
                        output_path = openpose_data+cohort+trial+camera+video+'\\'+frame
    #                     print (output_path)
                        if not os.path.exists(output_path):
                            process_openpose(im_path, output_path, feet_bool)
                print ('Time to process video through OpenPose: ', time.time()-start_time)
    print (cohort, ' done!')

Time to process video through OpenPose:  3.2589833736419678
Time to process video through OpenPose:  4.014221429824829
Time to process video through OpenPose:  11.913872003555298
Time to process video through OpenPose:  3.853573799133301
Time to process video through OpenPose:  4.94185996055603
Time to process video through OpenPose:  3.789902925491333
Time to process video through OpenPose:  4.094545364379883
Time to process video through OpenPose:  3.0591557025909424
Time to process video through OpenPose:  4.711664438247681
Time to process video through OpenPose:  11.334444761276245
Time to process video through OpenPose:  9.739710569381714
Time to process video through OpenPose:  2.868140459060669
Time to process video through OpenPose:  4.096503973007202
Time to process video through OpenPose:  3.2903287410736084
Time to process video through OpenPose:  3.0425024032592773
Time to process video through OpenPose:  1.4566001892089844
Time to process video through OpenPose:  3.2619204